In [12]:
import pandas as pd
import numpy as np
import os

In [13]:

# Create Data directory if it doesn't exist
os.makedirs('Data', exist_ok=True)

# Load your USDINR data
file_path = 'Data/USDINR_day_2025-01-2025-09-Infer.csv'
# Skip the ticker row (row 1) and empty row (row 2)
data = pd.read_csv(file_path, 
                   header=0,           # First row contains column names
                   skiprows=[1, 2],    # Skip ticker and empty rows
                   index_col=0, 
                   parse_dates=True)

# Clean column names
data.columns = data.columns.str.strip()

# Technical Indicator Functions
def SMA(series, window):
    """Simple Moving Average"""
    return series.rolling(window=window).mean()

def EMA(series, window):
    """Exponential Moving Average"""
    return series.ewm(span=window, adjust=False).mean()

def RSI(series, window=14):
    """Relative Strength Index"""
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

def MACD(series, fast=12, slow=26, signal=9):
    """MACD Indicator"""
    ema_fast = EMA(series, fast)
    ema_slow = EMA(series, slow)
    macd_line = ema_fast - ema_slow
    signal_line = EMA(macd_line, signal)
    macd_hist = macd_line - signal_line
    return macd_line, signal_line, macd_hist

def BollingerBands(series, window=20, no_of_std=2):
    """Bollinger Bands"""
    sma = SMA(series, window)
    std = series.rolling(window=window).std()
    upper_band = sma + no_of_std * std
    lower_band = sma - no_of_std * std
    return upper_band, sma, lower_band

def ATR(high, low, close, window=14):
    """Average True Range"""
    tr1 = high - low
    tr2 = (high - close.shift()).abs()
    tr3 = (low - close.shift()).abs()
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return true_range.rolling(window=window).mean()

def Stochastic(high, low, close, k_window=14, d_window=3):
    """Stochastic Oscillator"""
    lowest_low = low.rolling(window=k_window).min()
    highest_high = high.rolling(window=k_window).max()
    k_percent = 100 * ((close - lowest_low) / (highest_high - lowest_low))
    d_percent = k_percent.rolling(window=d_window).mean()
    return k_percent, d_percent


In [14]:

# Extract OHLC data
close = data['Close']
high = data['High']
low = data['Low']
open_ = data['Open']

# Calculate all indicators
print("Calculating technical indicators...")

# Moving Averages
sma_20 = SMA(close, 20)
sma_50 = SMA(close, 50)
ema_12 = EMA(close, 12)
ema_26 = EMA(close, 26)

# Momentum Indicators
rsi_14 = RSI(close, 14)
macd, macd_signal, macd_hist = MACD(close)
stoch_k, stoch_d = Stochastic(high, low, close)

# Volatility Indicators
upper_bb, middle_bb, lower_bb = BollingerBands(close)
atr_14 = ATR(high, low, close)

# Price-based Features
price_change = close.pct_change()
volatility_20 = close.rolling(20).std()
high_low_ratio = high / low

# Create processed dataset
processed_data = data.copy()

# Add all indicators
processed_data['SMA_20'] = sma_20
processed_data['SMA_50'] = sma_50
processed_data['EMA_12'] = ema_12
processed_data['EMA_26'] = ema_26
processed_data['RSI_14'] = rsi_14
processed_data['MACD'] = macd
processed_data['MACD_signal'] = macd_signal
processed_data['MACD_hist'] = macd_hist
processed_data['Stoch_K'] = stoch_k
processed_data['Stoch_D'] = stoch_d
processed_data['BB_upper'] = upper_bb
processed_data['BB_middle'] = middle_bb
processed_data['BB_lower'] = lower_bb
processed_data['ATR_14'] = atr_14
processed_data['Price_Change'] = price_change
processed_data['Volatility_20'] = volatility_20
processed_data['HL_Ratio'] = high_low_ratio

# Add forex-specific features
processed_data['Hour'] = processed_data.index.hour
processed_data['DayOfWeek'] = processed_data.index.dayofweek
processed_data['Month'] = processed_data.index.month

# Remove rows with NaN values (first 50 rows due to indicators)
processed_data_clean = processed_data.dropna()


Calculating technical indicators...


In [17]:

print(f"Original data shape: {data.shape}")
print(f"Processed data shape: {processed_data_clean.shape}")
print(f"Indicators added: {len(processed_data_clean.columns) - len(data.columns)}")

# Save processed data
output_path = 'Data/USDINR_day_2025-01-2025-09-Infer-processed.csv'
processed_data_clean.to_csv(output_path)

print(f"Processed data saved to: {output_path}")
print("\nFirst few rows of processed data:")
processed_data_clean.head()


Original data shape: (181, 5)
Processed data shape: (132, 25)
Indicators added: 20
Processed data saved to: Data/USDINR_day_2025-01-2025-09-Infer-processed.csv

First few rows of processed data:


,Close,High,Low,Open,Volume,SMA_20,SMA_50,EMA_12,EMA_26,RSI_14,...,BB_upper,BB_middle,BB_lower,ATR_14,Price_Change,Volatility_20,HL_Ratio,Hour,DayOfWeek,Month
Price,,,,,,,,,,,,,,,,,,,,,
2025-03-12,87.200104,87.304901,87.098801,87.200104,0,86.978815,86.694072,87.125654,86.982646,55.972867,...,87.588732,86.978815,86.368899,0.396692,-0.001774,0.304958,1.002366,0,2,3
2025-03-13,87.340599,87.340599,86.928596,87.340599,0,87.002145,86.725170,87.158723,87.009161,68.086037,...,87.630595,87.002145,86.373696,0.395664,0.001611,0.314225,1.004740,0,3,3
2025-03-14,86.922096,86.995697,86.866302,86.922096,0,87.010545,86.747532,87.122319,87.002711,57.014634,...,87.629453,87.010545,86.391637,0.408121,-0.004792,0.309454,1.001490,0,4,3
2025-03-17,86.939003,86.969803,86.690498,86.939003,0,87.035765,86.771132,87.094116,86.997992,54.686871,...,87.593988,87.035765,86.477543,0.376428,0.000195,0.279111,1.003222,0,0,3
2025-03-18,86.916397,86.918098,86.512901,86.916397,0,87.038845,86.795846,87.066775,86.991948,46.396674,...,87.593532,87.038845,86.484158,0.371243,-0.000260,0.277343,1.004684,0,1,3
